# Post-processing example

In [ ]:
#for more advanced hisfile operations: https://github.com/Deltares/dfm_tools/blob/main/tests/examples/postprocess_hisnc.py
#for more advanced mapfile operations: https://github.com/Deltares/dfm_tools/blob/main/tests/examples/postprocess_mapnc_ugrid.py
# -*- coding: utf-8 -*-
"""
Created on Fri Dec 6, 12:00
@author: Lorinc Meszaros, Deltares, Edito Model Lab
"""

#Import packages
import os
import matplotlib.pyplot as plt
import contextily as ctx
import numpy as np
import dfm_tools as dfmt
from matplotlib.colors import ListedColormap
import geopandas as gpd


In [ ]:
##Input
#file_nc=r'p:\11209182-edito-internship\HabitatSuitability\D-EcoImpact_results\Datasets\Eelgrass\min_HSI_combine_rule\ZosteraMarina*.nc'
file_nc=      

S3_ENDPOINT_URL = os.environ["S3_ENDPOINT"]
    fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
    onxia_user_name = os.environ["GIT_USER_NAME"] #TODO: convenient to have the username in a separate env var
    bucket_name = f"oidc-{onxia_user_name}"

##Output for plots
out=bucket_name
#%%

#Open map file
print('processing %s'%(os.path.basename(file_nc)))
uds_map = dfmt.open_partitioned_dataset(file_nc)
vars_pd = dfmt.get_ncvarproperties(uds_map)

#%%
#Settings for mapfile
crs = "EPSG:4326"
timestep=0
layer=0

#%%
#plot net/grid. use random variable and plot line to get grid
fig, ax = plt.subplots(figsize=(10,4))
pc = uds_map.grid.plot(edgecolor='crimson', linewidth=0.5)
if crs is None:
    ax.set_aspect('equal')
else:
    ctx.add_basemap(ax=ax, source=ctx.providers.Esri.WorldImagery, crs=crs, attribution=False)
#%%
# Create a custom colormap
cmap = plt.cm.Reds
cmap_with_transparency = cmap(np.arange(cmap.N))
cmap_with_transparency[0, -1] = 0  # Make the first color (corresponding to 0) fully transparent

# Create a new colormap from the modified array
transparent_cmap = ListedColormap(cmap_with_transparency)
#%%

#single plot
#uds_dpav=uds_map['Salinity'].isel(mesh2d_nLayers=layer,nmesh2d_layer=layer,missing_dims='ignore').min(dim="time") #aggregate
uds_dpav=uds_map['Total_HSI_MIN'].isel(time_month_year=timestep,mesh2d_nLayers=layer,nmesh2d_layer=layer,missing_dims='ignore')

fig, ax = plt.subplots(figsize=(10,4))
#range=[0.0,0.5]
pc=uds_dpav.ugrid.plot(cmap=transparent_cmap)
#pc.set_clim(range)
if crs is None:
    ax.set_aspect('equal')
else:
    ctx.add_basemap(ax=ax, source=ctx.providers.Esri.WorldImagery, crs=crs, attribution=False)

#%%
#Plot in loop

for variable in [
        # 'Temperature_2D_avg','IM2_Sediment_Concentration_2D', 
        # 'Total_Sediment_Concentration_2D','Fraction_Fine_Sediment_2D_avg',
        # 'Fraction_Fine_Sediment_2D', 'Water_Depth_avg', 'HSI_Water_Depth', 'HSI_Temperature','HSI_Fraction_Fine_Sediment',
        #'Orbital_Velocity_max', 'HSI_Orbital_Velocity' 
        'Salinity_2D_avg','HSI_Salinity',
        'Flow_Velocity_2D_max', 'HSI_Flow_Velocity',
        'Inundation_Time', 'HSI_Inundation_Time',
        'Total_HSI_MIN'
 ]:
    fig, ax = plt.subplots(figsize=(10,4))
    if vars_pd.loc[variable, 'shape'][0]>100:
        pc=uds_map[variable].isel(mesh2d_nLayers=layer,nmesh2d_layer=layer,missing_dims='ignore').mean(dim="time").ugrid.plot(cmap=transparent_cmap)
    elif vars_pd.loc[variable, 'shape'][0]>2 and vars_pd.loc[variable, 'shape'][0]<100:
        pc=uds_map[variable].isel(mesh2d_nLayers=layer,nmesh2d_layer=layer,missing_dims='ignore').mean(dim="time_month").ugrid.plot(cmap=transparent_cmap)
    else:
        pc = uds_map[variable].isel(time_month_year=timestep,mesh2d_nLayers=layer,nmesh2d_layer=layer,missing_dims='ignore').ugrid.plot(cmap=transparent_cmap)
    if crs is None:
        ax.set_aspect('equal')
    else:
        ctx.add_basemap(ax=ax, source=ctx.providers.Esri.WorldImagery, crs=crs, attribution=False)
    plt.savefig(out + '\\' + variable + '.png', bbox_inches='tight')
    print(variable)




In [ ]:
#%% Validation
# Load the validation data
shapefile_path = r'https://github.com/Deltares-research/EditoServices/blob/main/dfm-postprocess/validation-data/zostera_noltei_2017Polygon_WGS84.shp'
gdf = gpd.read_file(shapefile_path)

# Filter the GeoDataFrame for 'bedzostot' > 0
gdf_filtered = gdf[gdf['bedzostot'] > 0]

#Plot validation data
# Plot the specific column 'bedzostot'
fig, ax = plt.subplots(figsize=(10,4))
gdf_filtered.plot(ax=ax, column='bedzostot', color='Red', edgecolor='Red')
if crs is None:
    ax.set_aspect('equal')
else:
    ctx.add_basemap(ax=ax, source=ctx.providers.Esri.WorldImagery, crs=crs, attribution=False)

In [ ]:
# Set the title of the figure
ax.set_title("Observed seagreass (Zostera Noltii and littoral Zostera Marina) in 2017", fontsize=12) 

plt.show()

#%%
#Combined plot, Creating a figure with subplots
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(10, 10))

# Plot model results in the first subplot
range=[0.1,0.5]
pc = uds_map['Total_HSI_MIN'].isel(time_month_year=timestep,mesh2d_nLayers=layer,nmesh2d_layer=layer,missing_dims='ignore').ugrid.plot(ax=axs[0], cmap=transparent_cmap)
pc.set_clim(range)
if crs is None:
    ax.set_aspect('equal')
else:
    ctx.add_basemap(ax=axs[0], source=ctx.providers.Esri.WorldImagery, crs=crs, attribution=False)

# Set the title of the figure
axs[0].set_title("Simulated Habitat Suitability for ealgrass in the Dutch Wadden Sea in 2017", fontsize=12)


# Plot the GeoDataFrame in the second subplot
gdf_filtered.plot(ax=axs[1], column='bedzostot', color='Red', edgecolor='Red')

# Add basemap from contextily
ctx.add_basemap(axs[1], source=ctx.providers.Esri.WorldImagery, crs=crs, attribution=False)

# Set the title for the second subplot
axs[1].set_title("Observed seagreass (Zostera Noltii and littoral Zostera Marina) in 2017", fontsize=12)

# Show the figure
plt.tight_layout()
plt.savefig(out + '\\' + 'validation.png', bbox_inches='tight')
plt.show()